In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
plt.rcParams['figure.figsize'] = 10, 10

In [ ]:
filename = "tsopf_O0"
df0 = pd.read_csv(filename, sep=",", header=None, names=["algo", "sym_num", "threads", "time"], skipinitialspace=True,
                  dtype = {"algo": str, "sym_num": str, "threads": int, "time": float}, skipfooter=1, engine='python')
df0['sym_num'] = df0['sym_num'].astype(str)
print(df0.size)
df0.head()

In [ ]:
df = pd.DataFrame(df0.groupby(["algo", "sym_num", "threads"], as_index=False)["time"].mean()).sort_values("algo")
df

In [ ]:
data = {
    "algo": [],
    "symbolic": [],
    "numeric": []
}

for i, ((algo), group) in enumerate(df.groupby(["algo"])):
    data["algo"].append(algo)
    for j, ((symnum), subgroup) in enumerate(group.groupby(["sym_num"])):
        data[symnum].append(list(subgroup["time"])[0])

data

In [ ]:
multiplier = 1000000
for k in ["symbolic", "numeric"]:
    data[k] = list(map(lambda x: x*multiplier, data[k]))
data

In [ ]:
# normalize to simple
lsolve_simple_idx = data["algo"].index("lsolve_simple")
lsolve_simple_time = data["numeric"][lsolve_simple_idx]
data["numeric"] = [x / lsolve_simple_time for x in data["numeric"]]
data["symbolic"] = [x / lsolve_simple_time for x in data["symbolic"]]
data["algo"][data["algo"].index(f"lsolve_reachset_s_tsopf")] = "lsolve_reachset_tsopf"

In [ ]:

N = len(data["algo"])
x = np.arange(N)
w = 0.35

p1 = plt.bar(x, data["numeric"], w)
p2 = plt.bar(x, data["symbolic"], w, bottom=data["numeric"])

plt.title(f"Runtime for methods on {filename}")
plt.ylabel('Runtime (microseconds)')
plt.xticks(x, data["algo"])
plt.ylim(0, 1.2*max(data["numeric"] + data["symbolic"]))
plt.legend((p1[0], p2[0]), ('numeric', 'symbolic'))
plt.grid()

plt.show()


In [ ]:
# plt.savefig(f"../report/assets/runtime_{filename}.png", bbox_inches='tight')